In [1]:
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

# INPUT CONSTANTS
STEP_TAG = 'step2'
EXT_MODEL_TAG = 'eemd'
PATH_STEP2 = '/home/jaum/PG/pg/Final_Output/step2/eemd/'
# SCORE TAGS
SCORE_ACC_TAG = 'acc'#acc or f1
SCORE_F1_TAG = 'f1'
# CONSTANTS FROM CV CONFIG
CV_TAG = 'cv'
CV_TYPE_NORMAL_TAG = 'normal'
CV_TYPE_UNBIASED_TAG = 'unbiased'
CV_MODE_SIMPLE_TAG = 'simplegrid'
CV_MODE_NESTED_TAG = 'nested'
# CLF TAGS
CLF_KNN_TAG = 'knn'
CLF_RF_TAG = 'rf'
CLF_SVM_TAG = 'svm'
#
XML_TAG = '.xml'
RESULTS_TAG = 'results'
XML_SCORE_TAG = 'mean_score'
#tags
NS_TAG = 'Normal_Simple_{0}'
NN_TAG = 'Normal_Nested_{0}'
US_TAG = 'Unbiased_Simple_{0}'
UN_TAG = 'Unbiased_Nested_{0}'

In [2]:
def parse_unique_tag(file_in,tag,ret_attr):

    unique_tags = []
    for elem in file_in.iter(tag):
        return (float(elem.attrib[ret_attr]))

def get_classifier_files_path(clf_tag, score_tag):
    # files names
    ns_file = '{0}_{1}_{2}_{3}_{4}_{5}_{6}{7}'.format(STEP_TAG, EXT_MODEL_TAG, CV_TYPE_NORMAL_TAG,
                                                             CV_MODE_SIMPLE_TAG, clf_tag, score_tag, RESULTS_TAG, XML_TAG)
    nn_file = '{0}_{1}_{2}_{3}_{4}_{5}_{6}{7}'.format(STEP_TAG, EXT_MODEL_TAG, CV_TYPE_NORMAL_TAG,
                                                             CV_MODE_NESTED_TAG, clf_tag, score_tag, RESULTS_TAG, XML_TAG)
    us_file = '{0}_{1}_{2}_{3}_{4}_{5}_{6}{7}'.format(STEP_TAG, EXT_MODEL_TAG, CV_TYPE_UNBIASED_TAG,
                                                           CV_MODE_SIMPLE_TAG, clf_tag, score_tag, RESULTS_TAG, XML_TAG)
    un_file = '{0}_{1}_{2}_{3}_{4}_{5}_{6}{7}'.format(STEP_TAG, EXT_MODEL_TAG, CV_TYPE_UNBIASED_TAG,
                                                           CV_MODE_NESTED_TAG, clf_tag, score_tag, RESULTS_TAG, XML_TAG)
    # files path
    simple_path = '{0}{1}/{2}/'.format(PATH_STEP2, CV_MODE_SIMPLE_TAG, score_tag)
    nested_path = '{0}{1}/{2}/'.format(PATH_STEP2, CV_MODE_NESTED_TAG, score_tag)
    
    return simple_path + ns_file, nested_path + nn_file, simple_path + us_file, nested_path + un_file

def get_classifier_scores(clf_tag, score_tag):
    files_path = get_classifier_files_path(clf_tag, score_tag)
    
    clf_scores = []
    for file_path in files_path:
        file_iter = ET.parse(file_path)
        clf_scores.append(parse_unique_tag(file_iter, CV_TAG, XML_SCORE_TAG))
    
    return clf_scores

def get_df(clf_tags, score_tag):
    ss_index = [NS_TAG, NN_TAG, US_TAG, UN_TAG]
    ss_index = [tag.format(score_tag.title()) for tag in ss_index]
    
    clf_scores_ss = []
    for clf_tag in clf_tags:
        clf_scores = get_classifier_scores(clf_tag, score_tag)
        clf_scores_ss.append(pd.Series(clf_scores, index=ss_index))
    
    return pd.DataFrame([clf_ss for clf_ss in clf_scores_ss], index= ['Grid_KNN', 'Grid_RandomForest', 'Grid_SVM'])
    

In [3]:
# STEP2 Results for score ACCURACY using the EEMD as feature extraction model
get_df([CLF_KNN_TAG, CLF_RF_TAG, CLF_SVM_TAG], SCORE_ACC_TAG)

,Normal_Simple_Acc,Normal_Nested_Acc,Unbiased_Simple_Acc,Unbiased_Nested_Acc
Grid_KNN,0.979974,0.979815,0.775771,0.752060
Grid_RandomForest,0.999109,0.998306,0.903067,0.883317
Grid_SVM,0.993161,0.993161,0.864240,0.843518


In [5]:
# STEP2 Results for score F1_MACRO using the EEMD as feature extraction model
get_df([CLF_KNN_TAG, CLF_RF_TAG, CLF_SVM_TAG], SCORE_F1_TAG)

,Normal_Simple_F1,Normal_Nested_F1,Unbiased_Simple_F1,Unbiased_Nested_F1
Grid_KNN,0.972740,0.972505,0.509040,0.478222
Grid_RandomForest,0.998860,0.997765,0.740823,0.700914
Grid_SVM,0.990819,0.990819,0.626559,0.591448
